<a href="https://colab.research.google.com/github/yubars/ML-and-Cybersecurity-projects/blob/main/Machine_Learning_with_event_data_stream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Online Learning using Apache Kafka
#### Using Apache Kafka to simulate real time data stream model training.

### Learning Objecives
* Learn to import kafka and write to and read from kafka.
* Learn to use layers in keras.
* Learn to use kafka for data streams and train the model in real-time.
* Learn the concept of online learning.



### Install Package
Install Kafka and Tensorflow IO. Tensorflow IO is an extension package to Tensorflow which supports integration with Apache Kafka and other systems. 

In [ ]:
!pip install tensorflow-io
!pip install kafka-python

### Imports
Import all the required libraries for the lab including Kafka, pandas, Tensorflow, and more.



In [ ]:
import time 
from kafka import KafkaProducer
from sklearn.model_selection import train_test_split
import pandas as pd 
import tensorflow as tf
import tensorflow_io as tfio

### Download Kafka
Download and setup Kafka for real time data stream simulation. 



In [ ]:
!curl -sSOL https://downloads.apache.org/kafka/2.8.1/kafka_2.12-2.8.1.tgz
!tar -xzf kafka_2.12-2.8.1.tgz 

### Start running Kafka and Zookeeper server instances 

Start Kafka and Zookeeper servers as a daemon processes. Zookeeper is a centralized service for maintaing configuration information, naming, providing distributed synchronization, and providing group services.

In [ ]:
!./kafka_2.12-2.8.1/bin/zookeeper-server-start.sh -daemon ./kafka_2.12-2.8.1/config/zookeeper.properties
!./kafka_2.12-2.8.1/bin/kafka-server-start.sh -daemon ./kafka_2.12-2.8.1/config/server.properties

### Create a topic to store events
Create topic for train and test dataset to store events in Kafka. Kafka is a distributed event streaming platform that lets you read, write, store and process events. These events or messages are organized and stored in topics. In simple terms, topic is similar to a folder in a filesystem, and the events are the file in that folder.

In [ ]:
!./kafka_2.12-2.8.1/bin/kafka-topics.sh --create --topic susy-train --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1
!./kafka_2.12-2.8.1/bin/kafka-topics.sh --create --topic susy-test --bootstrap-server localhost:9092 --replication-factor 1 --partitions 2

Error while executing topic command : Topic 'susy-train' already exists.
[2021-10-29 14:40:03,807] ERROR org.apache.kafka.common.errors.TopicExistsException: Topic 'susy-train' already exists.
 (kafka.admin.TopicCommand$)
Error while executing topic command : Topic 'susy-test' already exists.
[2021-10-29 14:40:06,842] ERROR org.apache.kafka.common.errors.TopicExistsException: Topic 'susy-test' already exists.
 (kafka.admin.TopicCommand$)


### Describe the topic for details
Describe command helps us gather details on topic, it's partitions, replicas, and other important information.


In [ ]:
!./kafka_2.12-2.8.1/bin/kafka-topics.sh --describe --topic susy-train --bootstrap-server localhost:9092
!./kafka_2.12-2.8.1/bin/kafka-topics.sh --describe --topic susy-test --bootstrap-server localhost:9092

Topic: susy-train	TopicId: gJ-7Te6ESm2yQRMnIUTfoA	PartitionCount: 1	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: susy-train	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
Topic: susy-test	TopicId: iuLcF4x4R52j-d14TfVf6g	PartitionCount: 2	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: susy-test	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
	Topic: susy-test	Partition: 1	Leader: 0	Replicas: 0	Isr: 0


### Mount Google Drive

In the code cell below, we mount the google drive to the colab environment so that we have access to the local version of the dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Define features
COLUMNS is used to define each of the feature in the SUSY dataset.

In [ ]:
COLUMNS = [
           'class',
           'lepton_1_pT',
           'lepton_1_eta',
           'lepton_1_phi',
           'lepton_2_pT',
           'lepton_2_eta',
           'lepton_2_phi',
           'missing_energy_magnitude',
           'missing_energy_phi',
           'MET_rel',
           'axial_MET',
           'M_R',
           'M_TR_2',
           'R',
           'MT2',
           'S_R',
           'M_Delta_R',
           'dPhi_r_b',
           'cos(theta_r1)']

### SUSY Dataset

SUSY Data set is produced using Monte Carlo simulations. It is the data produced from the particle accelerators. The first column of the dataset is the label followed by 8 features which are kinematic properties measured by the particle detectors in the accelerator. The last 10 features are the high-level features derived by physicists to help discriminate between the two classes signal process or a background process.  

### Read CSV
Use Pandas to load SUSY dataset from the CSV file and provide the column name for each of the features.

In [ ]:
mydata = pd.read_csv('/content/drive/My Drive/Intro2MLDatasets/Lab6/SUSY.csv', header=None, names=COLUMNS, nrows=100000)

### Visualize Data

Panda functions helps us visualize the SUSY dataset. 

In [ ]:
mydata.head()

,class,lepton_1_pT,lepton_1_eta,lepton_1_phi,lepton_2_pT,lepton_2_eta,lepton_2_phi,missing_energy_magnitude,missing_energy_phi,MET_rel,axial_MET,M_R,M_TR_2,R,MT2,S_R,M_Delta_R,dPhi_r_b,cos(theta_r1)
0,0.0,0.972861,0.653855,1.176225,1.157156,-1.739873,-0.874309,0.567765,-0.175000,0.810061,-0.252552,1.921887,0.889637,0.410772,1.145621,1.932632,0.994464,1.367815,0.040714
1,1.0,1.667973,0.064191,-1.225171,0.506102,-0.338939,1.672543,3.475464,-1.219136,0.012955,3.775174,1.045977,0.568051,0.481928,0.000000,0.448410,0.205356,1.321893,0.377584
2,1.0,0.444840,-0.134298,-0.709972,0.451719,-1.613871,-0.768661,1.219918,0.504026,1.831248,-0.431385,0.526283,0.941514,1.587535,2.024308,0.603498,1.562374,1.135454,0.180910
3,1.0,0.381256,-0.976145,0.693152,0.448959,0.891753,-0.677328,2.033060,1.533041,3.046260,-1.005285,0.569386,1.015211,1.582217,1.551914,0.761215,1.715464,1.492257,0.090719
4,1.0,1.309996,-0.690089,-0.676259,1.589283,-0.693326,0.622907,1.087562,-0.381742,0.589204,1.365479,1.179295,0.968218,0.728563,0.000000,1.083158,0.043429,1.154854,0.094859


### Split Train and Test data
As always, it is necessary to split the data into train, test, and validation. In this context, we are splitting 80% of the data to be the train data. The remaining 20% of the data is split between test and validation dataset.

In [ ]:
train_df, test_df = train_test_split(mydata, test_size=0.3, shuffle=True)
print("Number of training samples: ",len(train_df))
print("Number of testing sample: ",len(test_df))

x_train_df = train_df.drop(["class"], axis=1)
y_train_df = train_df["class"]

x_test_df = test_df.drop(["class"], axis=1)
y_test_df = test_df["class"]

Number of training samples:  70000
Number of testing sample:  30000


### Convert data to list format
Read each row from the dataframe and convert it to the list format to feed to Kafka.

In [ ]:
#Convert each test and train dataframe to list form to feed to kafka
x_train = list(filter(None, x_train_df.to_csv(index=False).split("\n")[1:]))
y_train = list(filter(None, y_train_df.to_csv(index=False).split("\n")[1:]))

x_test = list(filter(None, x_test_df.to_csv(index=False).split("\n")[1:]))
y_test = list(filter(None, y_test_df.to_csv(index=False).split("\n")[1:]))
len(x_train), len(y_train), len(x_test), len(y_test)

(70000, 70000, 30000, 30000)

In [ ]:
NUM_COLUMNS = len(x_train_df.columns)

### Create Kafka Producer 
Create Kafka producer which takes in data and sends the record to the partition within a topic in Kafka cluster. 

In [ ]:
#send each record to a partition within a topic in kafka cluster
def write_to_kafka(topic, items):
  count=0
  producer = KafkaProducer(bootstrap_servers=['localhost:9092'])
  for message, key in items:
    producer.send(topic, key=key.encode('utf-8'), value=message.encode('utf-8'))
    count += 1 
  producer.flush()
  print("Wrote {0} messages into topic: {1}".format(count, topic))

write_to_kafka("susy-train", zip(x_train, y_train))
write_to_kafka("susy-test", zip(x_test, y_test))

Wrote 70000 messages into topic: susy-train
Wrote 30000 messages into topic: susy-test


### Online Learning
Unlike traditional training of machine learning models, online learning is based on incrementally learning or updating parameters as soon as the new data points are available. This process continues indefinitely. In the code below, stream_timeout is set to 10000 milliseconds which means as all the messages are consumed from the topic, the dataset will wait for 10 more seconds before timing out and disconnecting from the Kafka cluster. If additional data arrives in that time period, model training resumes. 

In [ ]:
online_train_ds = tfio.experimental.streaming.KafkaGroupIODataset(
    topics=["susy-train"],
    group_id="cgonline",
    servers="localhost:9092",
    stream_timeout=10000, # in milliseconds, to block indefinitely, set it to -1.
    configuration=[
        "session.timeout.ms=7000",
        "max.poll.interval.ms=8000",
        "auto.offset.reset=earliest"
    ],
)

def decode_kafka_online_item(raw_message, raw_key):
  message = tf.io.decode_csv(raw_message, [[0.0] for i in range(NUM_COLUMNS)])
  key = tf.strings.to_number(raw_key)
  return (message, key)

BATCH_SIZE = 32
online_train_ds = online_train_ds.shuffle(buffer_size=32)
online_train_ds = online_train_ds.map(decode_kafka_online_item)
online_train_ds = online_train_ds.batch(BATCH_SIZE)

### Initialize variables to create ANN
Initialize Optimizer, loss function, metrics function, and epochs for Neural Network.


In [ ]:
OPTIMIZER="adam"
LOSS = tf.keras.losses.BinaryCrossentropy()
METRICS = ['accuracy']
EPOCHS = 1

### Define Model
Create Neural network containing multiple layers and use Dropout layer to prevent overfitting in model. 

In [ ]:
#define model input shape, and layers of NN
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(NUM_COLUMNS,), activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 128)               2432      
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 256)               33024     
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                

In [ ]:
model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)

In [ ]:
model.fit(online_train_ds, epochs=EPOCHS)

2188/2188 [==============================] - 32s 15ms/step - loss: 0.4747 - accuracy: 0.7767


### Prepare test data

Prepare the test dataset using KafkaGroupIODataset to stream and test with the model we initialized before.

In [ ]:
#prepare test data to evaluate on the online trained model
test_ds = tfio.experimental.streaming.KafkaGroupIODataset(
    topics=["susy-test"],
    group_id="testcg",
    servers="localhost:9092",
    stream_timeout=10000,
    configuration=[
        "session.timeout.ms=7000",
        "max.poll.interval.ms=8000",
        "auto.offset.reset=earliest"
    ],
)

def decode_kafka_test_item(raw_message, raw_key):
  message = tf.io.decode_csv(raw_message, [[0.0] for i in range(NUM_COLUMNS)])
  key = tf.strings.to_number(raw_key)
  return (message, key)

test_ds = test_ds.map(decode_kafka_test_item)
test_ds = test_ds.batch(32)

### Evaluate Model
Use predefined function evaluate to figure loss value and metric value with the test data.

In [ ]:
res = model.evaluate(test_ds)

938/938 [==============================] - 23s 25ms/step - loss: 0.4498 - accuracy: 0.7917


loss: 0.4488 - accuracy: 0.7918
loss: 0.4476 - accuracy: 0.7905

In [ ]:
!./kafka_2.12-2.8.1/bin/kafka-consumer-groups.sh --bootstrap-server localhost:9092 --describe --group cgonline
!./kafka_2.12-2.8.1/bin/kafka-consumer-groups.sh --bootstrap-server localhost:9092 --describe --group testcg


Consumer group 'cgonline' has no active members.

GROUP           TOPIC           PARTITION  CURRENT-OFFSET  LOG-END-OFFSET  LAG             CONSUMER-ID     HOST            CLIENT-ID
cgonline        susy-train      0          280000          280000          0               -               -               -

Consumer group 'testcg' has no active members.

GROUP           TOPIC           PARTITION  CURRENT-OFFSET  LOG-END-OFFSET  LAG             CONSUMER-ID     HOST            CLIENT-ID
testcg          susy-test       0          64768           64768           0               -               -               -
testcg          susy-test       1          55232           55232           0               -               -               -


**What is the difference between this Lab (Lab 6) and Lab 4 (IDS)? [5 points]**


There are following differences, I observed,  between Lab 4 and this lab (Lab 6)

1.   In this lab, we train the model in real-time from live data streams but in Lab 4, we trained from historical dataset
2.   In this lab, We use a number of layers from keras (deep neural network) to build a model whereas we had used the Random Forest classifier in Lab 4
3.   In this lab, we installed an Apache Kafka server  and trained a model for online learning which is different from the general model that we developed in Lab4 in the sense that it incrementally trains model and update parameters when new data points (from stream) are available.
4.   We created columns heading for dataset in this lab and imported SUSY dataset in this lab whereas in Lab4, we imported KDD intrusion dataset with column heading from drive
5.   We had worked on an imbalanced dataset in Lab4 which was later made balanced with SMOTE methods but here we have not done so.







**Did you observe any differences in result during the evlaution of the model when you rerun? [5 points]**


####When I rerun the evaluate section only, it pops up an overflow error, meaning there is no more data in kafka server to read.
####But when I run all programs, it runs without significant difference in both evaluation metrics, loss and accuracy. The test loss/accuracy is almost the same as training loss/accuracy, so the model is performing well. While reruning with runall (entire programs), kafka server takes a new data stream, trains the model with those new data points, updates the parameters and evaluates for new test data so that offset value increases from previous. 